In [707]:
import os
import glob
import pandas as pd
import math
import numpy as np

In [708]:
files = glob.glob('whitehouse/hist-fy2019_copy2/*.csv')
# dfs2 = {}

# for fp in files:
#     dfs2.update({fp.split('.')[0].split('/')[2]: pd.read_csv(fp)})

In [709]:
def currency(col_val):
    currency = ''
    if "billion" in str(col_val): currency = 'billion'
    elif "million" in str(col_val): currency = 'million'
    else: currency = 'no currency'
    return currency

In [837]:
def process_file(norm):
    data_name = list(norm)[0].split(' -')[0] + '.csv'
    data_name = data_name.replace(" ", "_").replace(".", "_", 1).replace("Table", "t")
    # Detect the currency row
    norm['currency'] = norm.iloc[:,0].apply(currency)
    # Remove the currency row if present
    norm = norm.drop(norm.index[norm['currency']=='billion'].tolist())
    norm = norm.drop(norm.index[norm['currency']=='million'].tolist())
    norm = norm.reset_index(drop=True)
    # Remove the currency column
    norm = norm.drop(['currency'], axis=1)
    # Remove all \n
    norm = norm.replace(r'\n',' ', regex=True)
    # Replace all NaN with empty string
    norm = norm.replace(np.nan, '', regex=True)

    # This iterates over each row of the DF. 
    # Within each row, it replaces empty (previously NaN) values with the 
    # non-empty value to the left of the NaN.
    # Execution stops when the first value of a row 
    for index, row in norm.iterrows():
        if len(str(row[0])) == 4 and str(row[0])[0] == 1:
            pass
        for i in range(1, len(row)):
            if str(row[i-1]) == '': 
                pass
            else: 
                if str(row[i]) == '':
                    row[i] = row[i-1]
                    
    # Get the index where your header rows end:
    first = ''
    for i in norm.iloc[:,0]:
        if len(i) > 0 and i[0] == '1':
            first = i
            break

    top_index = norm.iloc[:,0][norm.iloc[:,0] == first].index[0]

    # Separate your headers rows (which you will compress) from your data rows
    norm_top = norm.iloc[0:top_index,:].copy()
    norm_bot = norm.iloc[top_index:,:].copy()
    
    # Compress only your header rows:
    tmp = norm_top.copy()

    # Add a dummy column that will serve as your index. Put it first.
    A = ['A'] * len(tmp)
    tmp.insert(loc=0, column='A', value=A)

    # Rename your columns to simply be strings '1' through 'x'
    new_col_names = range(0, len(list(tmp.columns)))
    new_col_names = map(str, new_col_names)
    tmp.columns = new_col_names

    # Take all but the first column
    cols_to_concat = list(tmp.columns[1:len(tmp.columns)])

    # Group by columns using column index and then drop it
    tmp = tmp.groupby(['0'], as_index = False)[cols_to_concat].agg('-'.join).drop(['0'], axis=1)
    
    # Need to rename the columns of norm_bot so that you can union with tmp:
    norm_bot.columns = list(tmp)

    # Re-attach your headers to the data in your dataframe:
    norm_final = pd.concat([tmp, norm_bot], axis=0)

    # Make the top row into your headers
    norm_final.columns = norm_final.iloc[0]
    
    # Drop first row and reset index:
    norm_final = norm_final.drop([0])
    norm_final = norm_final.reset_index(drop=True)

    # Write the DF to CSV using the name you extracted the name of the CSV
    norm_final.to_csv(data_name)
    print("wrote " + data_name + " to CSV!")
    
    return


In [838]:
# Full list (not all can be processed using process_file method):
files = ['whitehouse/hist-fy2019_copy2/hist01z1.csv','whitehouse/hist-fy2019_copy2/hist01z2.csv','whitehouse/hist-fy2019_copy2/hist01z3.csv','whitehouse/hist-fy2019_copy2/hist01z4.csv','whitehouse/hist-fy2019_copy2/hist02z1.csv','whitehouse/hist-fy2019_copy2/hist02z2.csv','whitehouse/hist-fy2019_copy2/hist02z3.csv','whitehouse/hist-fy2019_copy2/hist02z4.csv','whitehouse/hist-fy2019_copy2/hist02z5.csv','whitehouse/hist-fy2019_copy2/hist07z1.csv','whitehouse/hist-fy2019_copy2/hist07z2.csv','whitehouse/hist-fy2019_copy2/hist07z3.csv','whitehouse/hist-fy2019_copy2/hist08z1.csv','whitehouse/hist-fy2019_copy2/hist08z2.csv','whitehouse/hist-fy2019_copy2/hist08z3.csv','whitehouse/hist-fy2019_copy2/hist08z4.csv','whitehouse/hist-fy2019_copy2/hist08z5.csv','whitehouse/hist-fy2019_copy2/hist08z6.csv','whitehouse/hist-fy2019_copy2/hist08z7.csv','whitehouse/hist-fy2019_copy2/hist08z8.csv','whitehouse/hist-fy2019_copy2/hist09z1.csv','whitehouse/hist-fy2019_copy2/hist09z2.csv','whitehouse/hist-fy2019_copy2/hist09z3.csv','whitehouse/hist-fy2019_copy2/hist09z4.csv','whitehouse/hist-fy2019_copy2/hist09z5.csv','whitehouse/hist-fy2019_copy2/hist09z6.csv','whitehouse/hist-fy2019_copy2/hist09z7.csv','whitehouse/hist-fy2019_copy2/hist09z8.csv','whitehouse/hist-fy2019_copy2/hist09z9.csv','whitehouse/hist-fy2019_copy2/hist10z1.csv','whitehouse/hist-fy2019_copy2/hist11z1.csv','whitehouse/hist-fy2019_copy2/hist11z2.csv','whitehouse/hist-fy2019_copy2/hist11z3.csv','whitehouse/hist-fy2019_copy2/hist12z1.csv','whitehouse/hist-fy2019_copy2/hist12z2.csv','whitehouse/hist-fy2019_copy2/hist12z3.csv','whitehouse/hist-fy2019_copy2/hist14z1.csv','whitehouse/hist-fy2019_copy2/hist14z2.csv','whitehouse/hist-fy2019_copy2/hist14z3.csv','whitehouse/hist-fy2019_copy2/hist14z4.csv','whitehouse/hist-fy2019_copy2/hist14z5.csv','whitehouse/hist-fy2019_copy2/hist14z6.csv','whitehouse/hist-fy2019_copy2/hist15z1.csv','whitehouse/hist-fy2019_copy2/hist16z1.csv','whitehouse/hist-fy2019_copy2/hist16z2.csv']
# test_list:
test_files_to_process = ['whitehouse/hist-fy2019_copy2/hist01z1.csv','whitehouse/hist-fy2019_copy2/hist08z2.csv','whitehouse/hist-fy2019_copy2/hist02z1.csv','whitehouse/hist-fy2019_copy2/hist07z1.csv','whitehouse/hist-fy2019_copy2/hist08z1.csv']

# Final list of files to process:
final = ['whitehouse/hist-fy2019_copy2/hist01z1.csv','whitehouse/hist-fy2019_copy2/hist01z2.csv','whitehouse/hist-fy2019_copy2/hist01z3.csv','whitehouse/hist-fy2019_copy2/hist01z4.csv','whitehouse/hist-fy2019_copy2/hist02z1.csv','whitehouse/hist-fy2019_copy2/hist02z2.csv','whitehouse/hist-fy2019_copy2/hist02z3.csv','whitehouse/hist-fy2019_copy2/hist02z5.csv','whitehouse/hist-fy2019_copy2/hist07z1.csv','whitehouse/hist-fy2019_copy2/hist07z2.csv','whitehouse/hist-fy2019_copy2/hist08z1.csv','whitehouse/hist-fy2019_copy2/hist08z2.csv','whitehouse/hist-fy2019_copy2/hist08z3.csv','whitehouse/hist-fy2019_copy2/hist08z4.csv','whitehouse/hist-fy2019_copy2/hist09z1.csv','whitehouse/hist-fy2019_copy2/hist09z2.csv','whitehouse/hist-fy2019_copy2/hist09z3.csv','whitehouse/hist-fy2019_copy2/hist09z4.csv','whitehouse/hist-fy2019_copy2/hist09z5.csv','whitehouse/hist-fy2019_copy2/hist09z7.csv','whitehouse/hist-fy2019_copy2/hist10z1.csv','whitehouse/hist-fy2019_copy2/hist11z1.csv','whitehouse/hist-fy2019_copy2/hist12z1.csv','whitehouse/hist-fy2019_copy2/hist14z1.csv','whitehouse/hist-fy2019_copy2/hist14z2.csv','whitehouse/hist-fy2019_copy2/hist14z3.csv','whitehouse/hist-fy2019_copy2/hist14z4.csv','whitehouse/hist-fy2019_copy2/hist14z5.csv','whitehouse/hist-fy2019_copy2/hist14z6.csv','whitehouse/hist-fy2019_copy2/hist15z1.csv']

In [839]:
for fp in files:
    if fp in final:
        #bill = pd.read_csv(fp)
        #break
        process_file(pd.read_csv(fp))
        

wrote t_1_1.csv to CSV!
wrote t_1_2.csv to CSV!
wrote t_1_3.csv to CSV!
wrote t_1_4.csv to CSV!
wrote t_2_1.csv to CSV!
wrote t_2_2.csv to CSV!
wrote t_2_3.csv to CSV!
wrote t_2_5.csv to CSV!
wrote t_7_1.csv to CSV!
wrote t_7_2.csv to CSV!
wrote t_8_1.csv to CSV!
wrote t_8_2.csv to CSV!
wrote t_8_3.csv to CSV!
wrote t_8_4.csv to CSV!
wrote t_9_1.csv to CSV!
wrote t_9_2.csv to CSV!
wrote t_9_3.csv to CSV!
wrote t_9_4.csv to CSV!
wrote t_9_5.csv to CSV!
wrote t_9_7.csv to CSV!
wrote t_10_1.csv to CSV!
wrote t_11_1.csv to CSV!
wrote t_12_1.csv to CSV!
wrote t_14_1.csv to CSV!
wrote t_14_2.csv to CSV!
wrote t_14_3.csv to CSV!
wrote t_14_4.csv to CSV!
wrote t_14_5.csv to CSV!
wrote t_14_6.csv to CSV!
wrote t_15_1.csv to CSV!


#### Below this line is code for debugging/testing

In [798]:
# Choose only your CSVs where you don't need to transpose

# Grab 1 df to work with
norm = dfs2['hist01z2'].head(10)
#norm = dfs2['hist08z2'].head(10)
# vert = dfs2['hist04z2'].head(10)

# Extract name:
data_name = list(norm)[0].split(' -')[0] + '.csv'

In [801]:
# The first row in your dataframe usually just specifies the denomination of $.
# Remove this row if present.

def currency(col_val):
    currency = ''
    if "billion" in str(col_val): currency = 'billion'
    elif "million" in str(col_val): currency = 'million'
    else: currency = 'no currency'
    return currency

# Detect the currency row
# TODO: NEED TO RESET THE INDEX AFTER THIS STEP!
norm['currency'] = norm.iloc[:,0].apply(currency)
# Remove the currency row
norm = norm.drop(norm.index[norm['currency']=='billion'].tolist())
# Remove the currency column
norm = norm.drop(['currency'], axis=1)

/Users/ojasklowski/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [802]:
# Remove all \n
norm = norm.replace(r'\n',' ', regex=True)
# Replace all NaN with empty string
norm = norm.replace(np.nan, '', regex=True)

In [803]:
#norm

In [804]:
# This iterates over each row of the DF. 
# Within each row, it replaces empty (previously NaN) values with the 
# non-empty value to the left of the NaN.
# Execution stops when the first value of a row 
for index, row in norm.iterrows():
    if len(str(row[0])) == 4 and str(row[0])[0] == 1:
        pass
    for i in range(1, len(row)):
        if str(row[i-1]) == '': 
            pass
        else: 
            if str(row[i]) == '':
                row[i] = row[i-1]


In [805]:
#norm

In [806]:
norm = norm.reset_index(drop=True)
#norm

In [807]:
# Get the index where your header rows end:
first = ''
for i in norm.iloc[:,0]:
    if len(i) > 0 and i[0] == '1':
        first = i
        break

top_index = norm.iloc[:,0][norm.iloc[:,0] == first].index[0]

# Separate your headers rows (which you will compress) from your data rows
norm_top = norm.iloc[0:top_index,:].copy()
norm_bot = norm.iloc[top_index:,:].copy()

In [809]:
#norm_top

In [810]:
#norm_bot

In [811]:
# Compress only your header rows:
tmp = norm_top.copy()

In [812]:
# Add a dummy column that will serve as your index. Put it first.
A = ['A'] * len(tmp)
tmp.insert(loc=0, column='A', value=A)

# Rename your columns to simply be strings '1' through 'x'
new_col_names = range(0, len(list(tmp.columns)))
new_col_names = map(str, new_col_names)
tmp.columns = new_col_names

# Take all but the first column
cols_to_concat = list(tmp.columns[1:len(tmp.columns)])

# Group by columns using column index and then drop it
tmp = tmp.groupby(['0'], as_index = False)[cols_to_concat].agg('-'.join).drop(['0'], axis=1)

In [813]:
# Need to rename the columns of norm_bot so that you can union with tmp:
norm_bot.columns = list(tmp)

# Re-attach your headers to the data in your dataframe:
norm_final = pd.concat([tmp, norm_bot], axis=0)

# Make the top row into your headers
norm_final.columns = norm_final.iloc[0]
# Drop first row:
norm_final = norm_final.drop([0])

# Write the DF to CSV using the name you extracted the name of the CSV
norm_final.to_csv(data_name)

In [821]:
#norm_final
data_name

'Table 1.2.csv'

# Moving on to transpose data

In [847]:
# Full list (not all can be processed using process_file method):
files = ['whitehouse/hist-fy2019_copy2/hist01z1.csv','whitehouse/hist-fy2019_copy2/hist01z2.csv','whitehouse/hist-fy2019_copy2/hist01z3.csv','whitehouse/hist-fy2019_copy2/hist01z4.csv','whitehouse/hist-fy2019_copy2/hist02z1.csv','whitehouse/hist-fy2019_copy2/hist02z2.csv','whitehouse/hist-fy2019_copy2/hist02z3.csv','whitehouse/hist-fy2019_copy2/hist02z4.csv','whitehouse/hist-fy2019_copy2/hist02z5.csv','whitehouse/hist-fy2019_copy2/hist07z1.csv','whitehouse/hist-fy2019_copy2/hist07z2.csv','whitehouse/hist-fy2019_copy2/hist07z3.csv','whitehouse/hist-fy2019_copy2/hist08z1.csv','whitehouse/hist-fy2019_copy2/hist08z2.csv','whitehouse/hist-fy2019_copy2/hist08z3.csv','whitehouse/hist-fy2019_copy2/hist08z4.csv','whitehouse/hist-fy2019_copy2/hist08z5.csv','whitehouse/hist-fy2019_copy2/hist08z6.csv','whitehouse/hist-fy2019_copy2/hist08z7.csv','whitehouse/hist-fy2019_copy2/hist08z8.csv','whitehouse/hist-fy2019_copy2/hist09z1.csv','whitehouse/hist-fy2019_copy2/hist09z2.csv','whitehouse/hist-fy2019_copy2/hist09z3.csv','whitehouse/hist-fy2019_copy2/hist09z4.csv','whitehouse/hist-fy2019_copy2/hist09z5.csv','whitehouse/hist-fy2019_copy2/hist09z6.csv','whitehouse/hist-fy2019_copy2/hist09z7.csv','whitehouse/hist-fy2019_copy2/hist09z8.csv','whitehouse/hist-fy2019_copy2/hist09z9.csv','whitehouse/hist-fy2019_copy2/hist10z1.csv','whitehouse/hist-fy2019_copy2/hist11z1.csv','whitehouse/hist-fy2019_copy2/hist11z2.csv','whitehouse/hist-fy2019_copy2/hist11z3.csv','whitehouse/hist-fy2019_copy2/hist12z1.csv','whitehouse/hist-fy2019_copy2/hist12z2.csv','whitehouse/hist-fy2019_copy2/hist12z3.csv','whitehouse/hist-fy2019_copy2/hist14z1.csv','whitehouse/hist-fy2019_copy2/hist14z2.csv','whitehouse/hist-fy2019_copy2/hist14z3.csv','whitehouse/hist-fy2019_copy2/hist14z4.csv','whitehouse/hist-fy2019_copy2/hist14z5.csv','whitehouse/hist-fy2019_copy2/hist14z6.csv','whitehouse/hist-fy2019_copy2/hist15z1.csv','whitehouse/hist-fy2019_copy2/hist16z1.csv','whitehouse/hist-fy2019_copy2/hist16z2.csv']
# test_list:
test_list = ['whitehouse/hist-fy2019_copy2/hist04z2.csv']
# Final list of files to process:
to_transpose = ['whitehouse/hist-fy2019_copy2/hist02z4.csv','whitehouse/hist-fy2019_copy2/hist03z1.csv','whitehouse/hist-fy2019_copy2/hist03z2.csv','whitehouse/hist-fy2019_copy2/hist04z1.csv','whitehouse/hist-fy2019_copy2/hist04z2.csv','whitehouse/hist-fy2019_copy2/hist05z1.csv','whitehouse/hist-fy2019_copy2/hist05z2.csv','whitehouse/hist-fy2019_copy2/hist05z3.csv','whitehouse/hist-fy2019_copy2/hist05z4.csv','whitehouse/hist-fy2019_copy2/hist05z5.csv','whitehouse/hist-fy2019_copy2/hist05z6.csv','whitehouse/hist-fy2019_copy2/hist06z1.csv','whitehouse/hist-fy2019_copy2/hist08z5.csv','whitehouse/hist-fy2019_copy2/hist08z6.csv','whitehouse/hist-fy2019_copy2/hist08z7.csv','whitehouse/hist-fy2019_copy2/hist08z8.csv','whitehouse/hist-fy2019_copy2/hist09z6.csv','whitehouse/hist-fy2019_copy2/hist09z8.csv','whitehouse/hist-fy2019_copy2/hist09z9.csv','whitehouse/hist-fy2019_copy2/hist11z2.csv','whitehouse/hist-fy2019_copy2/hist11z3.csv','whitehouse/hist-fy2019_copy2/hist12z2.csv','whitehouse/hist-fy2019_copy2/hist12z3.csv','whitehouse/hist-fy2019_copy2/hist13z1.csv']

In [851]:
# for fp in files:
#     if fp in test_list:
#         #bill = pd.read_csv(fp)
#         #break
#         #transpose_file(pd.read_csv(fp))
#         yo = pd.read_csv(fp)

test_list[0]
yo = pd.read_csv(test_list[0])
yo

,Table 4.2 - PERCENTAGE DISTRIBUTION OF OUTLAYS BY AGENCY: 1962 - 2023,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63
0,Department or other unit,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2014,2015,2016,2017,2018 estimate,2019 estimate,2020 estimate,2021 estimate,2022 estimate,2023 estimate
1,Legislative Branch,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,Judicial Branch,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
3,Department of Agriculture,6.0,6.7,6.4,5.9,4.2,3.8,4.2,4.6,4.3,...,4.0,3.8,3.6,3.2,3.5,2.7,2.5,2.5,2.4,2.3
4,Department of Commerce,0.2,0.3,0.6,0.6,0.4,0.3,0.3,0.3,0.4,...,0.2,0.2,0.2,0.3,0.2,0.3,0.3,0.2,0.2,0.1
5,Department of Defense--Military Programs,46.9,45.9,44.4,41.3,42.1,44.5,45.1,44.0,41.0,...,16.5,15.3,14.7,14.3,14.7,14.9,15.2,14.9,14.4,14.2
6,Department of Education,0.8,0.9,0.8,1.0,1.8,2.3,2.3,2.2,2.3,...,1.7,2.4,2.0,2.8,1.5,1.6,1.4,1.2,1.2,1.1
7,Department of Energy,2.6,2.4,2.3,2.2,1.7,1.4,1.4,1.3,1.2,...,0.7,0.7,0.7,0.6,0.7,0.6,0.6,0.6,0.6,0.5
8,Department of Health and Human Services,3.3,3.7,3.9,4.0,4.2,6.1,7.3,8.4,8.9,...,26.7,27.9,28.6,28.0,28.0,27.4,26.1,25.9,26.8,26.7
9,Department of Homeland Security,0.5,0.6,0.6,0.7,0.5,0.5,0.5,0.6,0.6,...,1.2,1.2,1.2,1.3,2.0,1.3,1.2,1.2,1.1,1.1


In [ ]:
data_name = list(norm)[0].split(' -')[0] + '.csv'
data_name = data_name.replace(" ", "_").replace(".", "_", 1).replace("Table", "t")
# Detect the currency row
norm['currency'] = norm.iloc[:,0].apply(currency)
# Remove the currency row if present
norm = norm.drop(norm.index[norm['currency']=='billion'].tolist())
norm = norm.drop(norm.index[norm['currency']=='million'].tolist())
norm = norm.reset_index(drop=True)
# Remove the currency column
norm = norm.drop(['currency'], axis=1)
# Remove all \n
norm = norm.replace(r'\n',' ', regex=True)
# Replace all NaN with empty string
norm = norm.replace(np.nan, '', regex=True)

# This iterates over each row of the DF. 
# Within each row, it replaces empty (previously NaN) values with the 
# non-empty value to the left of the NaN.
# Execution stops when the first value of a row 
for index, row in norm.iterrows():
    if len(str(row[0])) == 4 and str(row[0])[0] == 1:
        pass
    for i in range(1, len(row)):
        if str(row[i-1]) == '': 
            pass
        else: 
            if str(row[i]) == '':
                row[i] = row[i-1]

# Get the index where your header rows end:
first = ''
for i in norm.iloc[:,0]:
    if len(i) > 0 and i[0] == '1':
        first = i
        break

top_index = norm.iloc[:,0][norm.iloc[:,0] == first].index[0]

# Separate your headers rows (which you will compress) from your data rows
norm_top = norm.iloc[0:top_index,:].copy()
norm_bot = norm.iloc[top_index:,:].copy()

# Compress only your header rows:
tmp = norm_top.copy()

# Add a dummy column that will serve as your index. Put it first.
A = ['A'] * len(tmp)
tmp.insert(loc=0, column='A', value=A)

# Rename your columns to simply be strings '1' through 'x'
new_col_names = range(0, len(list(tmp.columns)))
new_col_names = map(str, new_col_names)
tmp.columns = new_col_names

# Take all but the first column
cols_to_concat = list(tmp.columns[1:len(tmp.columns)])

# Group by columns using column index and then drop it
tmp = tmp.groupby(['0'], as_index = False)[cols_to_concat].agg('-'.join).drop(['0'], axis=1)

# Need to rename the columns of norm_bot so that you can union with tmp:
norm_bot.columns = list(tmp)

# Re-attach your headers to the data in your dataframe:
norm_final = pd.concat([tmp, norm_bot], axis=0)

# Make the top row into your headers
norm_final.columns = norm_final.iloc[0]

# Drop first row and reset index:
norm_final = norm_final.drop([0])
norm_final = norm_final.reset_index(drop=True)

# Write the DF to CSV using the name you extracted the name of the CSV
norm_final.to_csv(data_name)
print("wrote " + data_name + " to CSV!")

return


['whitehouse/hist-fy2019_copy2/hist02z4.csv',
 'whitehouse/hist-fy2019_copy2/hist16z1.csv',
 'whitehouse/hist-fy2019_copy2/hist08z7.csv',
 'whitehouse/hist-fy2019_copy2/hist08z5.csv',
 'whitehouse/hist-fy2019_copy2/hist12z3.csv',
 'whitehouse/hist-fy2019_copy2/hist07z3.csv',
 'whitehouse/hist-fy2019_copy2/hist08z8.csv',
 'whitehouse/hist-fy2019_copy2/hist09z9.csv',
 'whitehouse/hist-fy2019_copy2/hist11z2.csv',
 'whitehouse/hist-fy2019_copy2/hist09z8.csv',
 'whitehouse/hist-fy2019_copy2/hist12z2.csv',
 'whitehouse/hist-fy2019_copy2/hist11z3.csv',
 'whitehouse/hist-fy2019_copy2/hist08z6.csv',
 'whitehouse/hist-fy2019_copy2/hist16z2.csv',
 'whitehouse/hist-fy2019_copy2/hist09z6.csv']